In [0]:
# import the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve
import warnings
warnings.filterwarnings('ignore')



In [0]:
pwd

'/Workspace/Users/youneslimoury6999@gmail.com/customer-churn-prediction'

In [0]:
import os
import pandas as pd

# تحديد مجلد notebook الحالي
current_path = os.getcwd()  # هذا يعطي المسار الحالي للـ notebook

# حساب المسار الكامل للملف باستخدام الرجوع بالمسار النسبي
target_path = os.path.abspath(os.path.join(
    current_path,
    "../../../data/customer_churn/WA_Fn-UseC_-Telco-Customer-Churn.csv"
))


# قراءة الملف باستخدام pandas
df = pd.read_csv(target_path)

# عرض أول صفين للتأكد من التحميل
df.head(2)


Full file path: /Workspace/data/customer_churn/WA_Fn-UseC_-Telco-Customer-Churn.csv


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No


Let's first clarify the variables of our dataset:

customerID – Unique identifier for each customer.

gender – Customer’s gender (e.g., Male/Female).

SeniorCitizen – Indicates if the customer is a senior (1 = Yes, 0 = No).

Partner – Whether the customer has a spouse/partner.

Dependents – Whether the customer has dependents (children, etc.).

tenure – Number of months the customer has stayed with the company.

PhoneService – Whether the customer has phone service.

MultipleLines – Whether the customer has multiple phone lines.

InternetService – Type of internet service (DSL, Fiber, None).

OnlineSecurity – Whether the customer has online security add-on.

OnlineBackup – Whether the customer has online backup add-on.

DeviceProtection – Whether the customer has device protection add-on.

TechSupport – Whether the customer has tech support add-on.

StreamingTV – Whether the customer has streaming TV service.

StreamingMovies – Whether the customer has streaming movie service.

Contract – Customer’s contract type (Month-to-month, 1-year, 2-year).

PaperlessBilling – Whether the customer uses paperless billing.

PaymentMethod – Customer’s payment method (e.g., credit card, bank transfer).

MonthlyCharges – Amount charged to the customer each month.

TotalCharges – Total amount charged over the entire tenure.

Churn – Indicates if the customer left the service (Yes/No).


In [0]:
# explore the data
df.shape

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-5202546002995263>, line 2
      1 # explore the data
----> 2 df.shape

NameError: name 'df' is not defined

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [0]:
# =======================================
# 1️⃣ Project Setup & Data Collection
# =======================================

# =======================================
# 2️⃣ Exploratory Data Analysis (EDA)
# =======================================

# =======================================
# 3️⃣ Data Cleaning & Preprocessing
# =======================================

# =======================================
# 4️⃣ Feature Engineering
# =======================================

# =======================================
# 5️⃣ Model Building & Training
# =======================================

# =======================================
# 6️⃣ Model Evaluation & Business Analysis
# =======================================

# =======================================
# 7️⃣ Model Explainability
# =======================================

# =======================================
# 8️⃣ Model Deployment (API)
# =======================================

# =======================================
# 9️⃣ User Interface / Dashboard (Optional)
# =======================================

# =======================================
# 🔟 Portfolio & Documentation
# =======================================
